# Homework 4: Modeling Text Data

### Team Member 1:
* UNI:  yo2258
* Name: Ye Ouyang

### Team Member 2 [optional]:
* UNI:  
* Name:


You can find the data here: https://data.boston.gov/dataset/vision-zero-entry

# Task1 - Data Cleaning  [10 points]

Load the data, visualize the class distribution. Clean up the target labels. Some categories have been arbitrarily split and need to be consolidated. 

In [20]:
# Add your code for task 1 here. You may use multiple cells. 

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from sklearn import metrics

#%% 1) Data Cleaning  

# Load data and Visualize the class distribution
reviews_train_raw = pd.read_csv ("Vision_Zero_Entry.csv")

# Clean up the target labels
reviews_train_raw = reviews_train_raw.drop('GLOBALID',axis=1)
reviews_train=reviews_train_raw.dropna()

# Class visulazation
fig, ax = plt.subplots()
reviews_train.REQUESTTYPE.value_counts().plot(ax=ax, kind='bar')

# Split training and testing data
text_train, y_train = reviews_train.COMMENTS.tolist(), reviews_train.REQUESTTYPE.tolist()
text_train_sub,text_val,y_train_sub,y_val = train_test_split(text_train,y_train,random_state=0)

y_train_sub=np.asarray(y_train_sub)
y_val=np.asarray(y_val)



# Task2 - Model 1 [10 points]

Run a baseline multi-class classification model using a bag-of-word approach, report macro f1-score (should be above .5) and visualize the confusion matrix. Can you interpret the mistakes made by the model? 

In [21]:
# Use SGDClassifier as classfication method, and get score as 0.543
# Confusion matrix is used to show the result

from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, n_iter=5, random_state=42))])
_ = text_clf.fit(text_train_sub, y_train_sub)
predicted = text_clf.predict(text_val)

print (np.mean(predicted == y_val)) 
print(metrics.classification_report(y_val, predicted))


# Task3 - Model 2 [30 points]

Improve the model using more complex text features, including n-grams, character n-grams and possibly domain-specific features.

In [22]:
# Set the parameters in CountVectorizer, TfidTransformer, and Classification method to improve the accuracy
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,3))),('tfidf', TfidfTransformer(use_idf=True)),('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, n_iter=5, random_state=42))])
_ = text_clf.fit(text_train_sub, y_train_sub)
predicted = text_clf.predict(text_val)

# Task4 - Visualize Results [10 points]

Visualize results of the tuned model (classification results, confusion matrix, important features, example mistakes).

In [23]:
# Show the classification result, confusion matrix and other information 
print (np.mean(predicted == y_val)) 
print(metrics.classification_report(y_val, predicted))


# Task5 - Clustering [10 points]

Apply LDA, NMF and K-Means to the whole dataset. Can you find clusters or topics that match well with some of the ground truth labels? Use ARI to compare the methods and visualize topics and clusters.

In [24]:
# Use K-Means to do the clustering, and set the number of cluster as 5

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(y_train)
true_k = 5
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)




# Task6 - Model 3 [30 points]

Improve the class definition for REQUESTTYPE by using the results of the clustering and results of the previous classification model. Re-assign labels using either the results of clustering or using keywords that you found during data exploration. The labels must be semantically meaningful.
The data has a large “other” category. Apply the topic modeling and clustering techniques to this subset of the data to find possible splits of this class.
Report accuracy using macro average f1 score (should be above .53) 


In [25]:
# Use the clustering result to label the data
# The improved method could highly improve the accuracy to 0.6369

def improvedModel():
    y_train = model.labels_
    text_train_sub,text_val,y_train_sub,y_val = train_test_split(text_train,y_train,random_state=0)

    y_train_sub=np.asarray(y_train_sub)
    y_val=np.asarray(y_val)

    text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,3))),('tfidf', TfidfTransformer(use_idf=True)),('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, n_iter=5, random_state=42))])
    _ = text_clf.fit(text_train_sub, y_train_sub)
    predicted = text_clf.predict(text_val)

    print (np.mean(predicted == y_val)) 
    print(metrics.classification_report(y_val, predicted))
    return np.mean(predicted == y_val)


def test_accuracy():
    print (improvedModel())
    assert(improvedModel()>0.62)



# Extra Credit [Up to +20 points]

Use a word embedding representation like word2vec for step 3 and or step 6. 

In [26]:
# Add your code for extra credit here. You may use multiple cells. 
